# Dynamische Arrays

Wir haben gesehen, wie wir die Methoden ```append``` und ```pop```, die ein Element am Ende eines Arrays anfügen, respektive entfernen effizient implementieren können. Hier nochmals die Implementation:

In [65]:
class  Array:

    def __init__(self):
        self.data = [None] # list  simulates  block  of  memory
        self.idx = 0
        
    def append(self, elem):
        if self.idx == len(self.data):            
            self.resize(len(self.data) * 2)
        self.data[self.idx] = elem
        self.idx  += 1        
    
    def  pop(self):
        self.idx -= 1
        item = self.data[self.idx]; 
        if self.idx > 0  and self.idx == len(self.data) / 4:
            self.resize(int(len(self.data) / 2));   

        return item; 

    def resize(self, numElements ):
        newArray = [None] * numElements

        for i in  range(0, len(self.data )):            
            newArray[i] = self.data[i]
        self.data = newArray
    
    def __str__(self):
        return str(self.data)
    
    def length(self):
        return self.idx


Die folgenden Tests zeigen, dass die Implementation wie erwartet funktioniert:

In [59]:
a = Array()
for i in range(0, 10):
    a.append(i)
print("Array after append: " + str(a))
print("length: " +str(a.length()))

lastElement = a.pop()
print("lastElement " + str(lastElement))
print("Array after pop: " +str(a))
print("length after pop: " +str(a.length()))


Array after append: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None, None, None, None, None, None]
length: 10
lastElement 9
Array after pop: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None, None, None, None, None, None]
length after pop: 9


#### Experimente

Wir wollen nun testen, wie sich die Laufzeit der append Methode verhält. 

In [60]:
import timeit
import random

In [61]:
def createByAppend(n):
    a = Array()
    for i in range(0, n):
        a.append(i)
    return a;

In [62]:
for n in [10, 100, 1000, 10000, 100000, 100000]:
 
    t = timeit.timeit(lambda: createByAppend(n), number=100)
    print("Durchschnittliche Zeit für eine append Operation bei " + str(n) + " Elementen = " + str(t / n))

Durchschnittliche Zeit für eine append Operation bei 10 Elementen = 0.00011670999992929865
Durchschnittliche Zeit für eine append Operation bei 100 Elementen = 7.10129999970377e-05
Durchschnittliche Zeit für eine append Operation bei 1000 Elementen = 5.424940000011702e-05
Durchschnittliche Zeit für eine append Operation bei 10000 Elementen = 4.836630999998306e-05
Durchschnittliche Zeit für eine append Operation bei 100000 Elementen = 4.472398599998997e-05
Durchschnittliche Zeit für eine append Operation bei 100000 Elementen = 4.683580599999914e-05


Wie wir sehen, ist die Durchschnittliche Zeit pro ```append```-Operation konstant, unabhängig davon, wieviele Elemente wir einfügen. 

##### Übungen: 

* Machen Sie dasselbe Experiment mit der pop Methode.
* Vergleichen Sie die Laufzeit mit einer naiven Implementation, bei der das Array bei jedem Aufruf vergrössert wird.

### Ammortisierte Analyse: Array resizing

Im Folgenden schauen wir uns die ```append``` Operation noch auf theoretischer Ebene an. Der folgende Code dient zum illustrieren der Idee der amortisierten Analyse. Die Idee ist einfach - wir führen Buchhalten. Bei jeder ```billigen``` Operation bezahlen einen Betrag, mit dem wir später die teure Operation bezahlen. Unsere Währung ist dabei ein *Token*. Für jeden Arrayzugriff in der resize Methode bezahlen wir mit einem Token. Wenn wir es schaffen, dass wir bei jeder ```append```-Operation immer dieselbe Konstante Anzahl Tokens auf die Seite legen können, und damit genügend Tokens zur Verfügung haben um damit die Anzahl Array-Zugriffe in der ```resize``` Operation bezahlen können, haben wir eine amortisiert konstante Laufzeit.

Im folgenden Code ist diese Idee umgesetzt:

In [120]:
class  ArrayWithAccounting:

    def __init__(self):
        self.data = [None] 
        self.idx = 0
        self.tokens = 0 # Anzahl Tokens ist am Anfang 0
        
    def append(self, elem):
        if self.idx == len(self.data):            
            self.resize(len(self.data) * 2)
        
        # Wir legen hier 8 Tokens auf die Seite, um für alle Array-Zugriffe in der resize Operation zu bezahlen
        self.tokens += 8
        
        self.data[self.idx] = elem
        
        self.idx  += 1        
        print("tokens after append " + str(self.tokens))

    def resize(self, numElements ):
        # Ein neues Array anzulegen brauch 1 Array Zugriffe pro Element
        # Wir bezahlen also ein Token pro Element
        newArray = [None] * numElements
        self.tokens -= 1 * numElements 
        
        for i in  range(0, len(self.data )):            
            newArray[i] = self.data[i]
            
            # Für jedes kopierte Element brauchen wir 2 Array Zugriffe 
            # Wir bezahlen also 2 Token pro Element
            self.tokens -= 2 
            
        self.data = newArray
    
    def __str__(self):
        return str(self.data)


Wenn wir diesen Code Ausführen, dann sehen wir, dass wir nach jeder Append Operation immer eine positive Anzahl Tokens haben. 

In [126]:
a = ArrayWithAccounting()
for i in range(0, 32):
    a.append(i)


tokens after append 8
tokens after append 12
tokens after append 12
tokens after append 20
tokens after append 12
tokens after append 20
tokens after append 28
tokens after append 36
tokens after append 12
tokens after append 20
tokens after append 28
tokens after append 36
tokens after append 44
tokens after append 52
tokens after append 60
tokens after append 68
tokens after append 12
tokens after append 20
tokens after append 28
tokens after append 36
tokens after append 44
tokens after append 52
tokens after append 60
tokens after append 68
tokens after append 76
tokens after append 84
tokens after append 92
tokens after append 100
tokens after append 108
tokens after append 116
tokens after append 124
tokens after append 132


*Übung: Würde es auch mit 7 Tokens funktionieren? Experimentieren Sie!*

*Übung: Können Sie ein ähnliches Experiment für die pop-Operation machen?*

#### Satz: Dynamisches vergrössern, verkleinern eines Arrays

Was wir nun experimentell herausgefunden haben lässt sich formal beweisen. Für eine Beweisskizze, siehe Sedgewick und Wayne, Algorithms, Satz E, Seite 221).

> Satz: Bei einem Array mit variabler Grösse ist gemäss obigen Algorithmus ist die durchschnittliche Anzahl der Arrayzugriffe für jede beliebige Folge von $M$ ```append``` und ```pop``` Operationen, ausgehend von dem leeren Array, proportional zu $M$.


### Dynamische Arrays in Python


Wie bereits im vorigen Notebook besprochen, ist ein Array in Python durch den Datentyp ```List``` implementiert. 
Der Datenbyp ```List``` entspricht einem dynamischen Array und unterstützt bereits die ```append``` und ```pop``` Methoden. Auch hier wollen wir wieder experimentell überprüfen, ob die Laufzeit unseren Erwartungen entspricht. 



In [133]:
def createByAppendPython(n):
    a = []
    for i in range(0, n):
        a.append(i)
    return a;

In [134]:
for n in [10, 100, 1000, 10000, 100000, 100000]:
 
    t = timeit.timeit(lambda: createByAppendPython(n), number=100)
    print("Durchschnittliche Zeit für eine append Operation bei " + str(n) + " Elementen = " + str(t / n))

Durchschnittliche Zeit für eine append Operation bei 10 Elementen = 2.0909999875584617e-05
Durchschnittliche Zeit für eine append Operation bei 100 Elementen = 9.523999979137444e-06
Durchschnittliche Zeit für eine append Operation bei 1000 Elementen = 1.3169500001822598e-05
Durchschnittliche Zeit für eine append Operation bei 10000 Elementen = 1.746640000019397e-05
Durchschnittliche Zeit für eine append Operation bei 100000 Elementen = 1.5012366000009933e-05
Durchschnittliche Zeit für eine append Operation bei 100000 Elementen = 1.6294950999981665e-05


Python stellt auch eine Methode zur Verfügung, um an einer beliebigen Stelle im Array ein Element einzufügen. Es ist leicht zu sehen, dass wir beim Einfügen an einer beliebigen Stelle, alle Elemente hinter der Einfügestelle kopieren müssen. Diese Operation hat also im schlimmsten Fall lineare Laufzeit in der Arraygrösse. Auch dies können wir einfach experimentell bestätigen

In [135]:
def insertFirstPython(n):
    a = []
    for i in range(0, n):
        a.insert(0, i)
    return a;

In [136]:
for n in [10, 100, 1000, 10000, 100000]:
 
    t = timeit.timeit(lambda: insertFirstPython(n), number=10)
    print("Durchschnittliche Zeit für eine append Operation bei " + str(n) + " Elementen = " + str(t / n))

Durchschnittliche Zeit für eine append Operation bei 10 Elementen = 3.5700002626981585e-06
Durchschnittliche Zeit für eine append Operation bei 100 Elementen = 2.127999978256412e-06
Durchschnittliche Zeit für eine append Operation bei 1000 Elementen = 6.078000002162298e-06
Durchschnittliche Zeit für eine append Operation bei 10000 Elementen = 4.094361999996181e-05
Durchschnittliche Zeit für eine append Operation bei 100000 Elementen = 0.0003399953269999969


*Übung: Machen Sie die selben Experimente mit der Methode ```pop```. Anmerkung: ```pop(i)``` entfernt das Element an Stelle i, ```pop()``` entfernt das letzte Element. Welches ist schneller?*